In [ ]:
!pip install yfinance
import pandas as pd 
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')

In [ ]:
def gapup(stock_name,start_date,end_date,Amount):
  data = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    progress=False)
  value=[]
  for i in zip(data['Open'],data['Close']):
    if i[0]>i[1]:
      value.append('Red')
    else:
      value.append('Green')
  data['color']= value
  sash=[]
  for i in range(0,len(data)):
    try:
      if (data['Close'][i] < data['Open'][i+1]) and (data['Open'][i] > data['Open'][i+1]) and (data['Open'][i] < data['Close'][i+1]) and (data['color'][i] == 'Red') and (data['color'][i+1] == 'Green'):
        sash.append('Bull Sash')
      else:
        sash.append('flat')
    except:
      sash.append('flat')
  data['Sash']= sash
  data['Sash']=data['Sash'].shift(1)
  indication=[]
  for i in range(0,len(data)-1):
    #if data['Sash'][i] == 'Bull Sash' and (data['Open'][i-1] - data['Close'][i-1]) <= (data['Close'][i] - data['Open'][i])  and data['Close'][i] <= data['Open'][i+1]:
    if data['Sash'][i] == 'Bull Sash' and data['Close'][i] <= data['Open'][i+1]:
      indication.append('Buy')
    else:
      indication.append('No')
  indication.append('No')

  # data['stop loss']=data[stop_loss].shift(2) # changes to Open from Close

  data['indication']=indication
  data['indication']=data['indication'].shift(1)
  # data=data.dropna()

  data.reset_index(inplace=True)

  # # ADD MIN DATA

  data2=yf.download(stock_name, 
                      start=start_date, 
                      end=end_date,
                      progress=False,interval='5m')

  #data2.dropna(inplace=True)
  data2.reset_index(inplace=True)
  # sep date time
  data2['Datetime']=data2['Datetime'].astype('str')
  df=data2['Datetime'].str.split(' ',expand=True)
  data2['Date']=df[0]
  data2['Time']=df[1]

  # NSE WAS CLOSE DUE TO GLICTH

  for i in data2['Date']:
    if i=='2021-02-24':
      data2.drop(data2[data2['Date'] == i].index, inplace = True) 

  data2.reset_index(inplace=True)
  data2.drop('index',axis=1,inplace=True)

  data=data.reset_index().drop('index',axis=1)

  data['Date']=data['Date'].astype('str')

  for i in data['Date']:
    if i=='2021-02-24':
      data.drop(data[data['Date'] == i].index, inplace = True) 

  data=data.reset_index().drop('index',axis=1)

  buy_data=data2.copy()

  # remove all other time data # selling
  for i in data2['Time']:
    if i=='14:55:00+05:30':
      pass
    else:
      data2.drop(data2[data2['Time'] == i].index, inplace = True) 
  data2=data2.reset_index().drop('index',axis=1)

  data['Close']=data2['Open']

  # remove all other time data # for buying
  for i in buy_data['Time']:
    if i=='09:20:00+05:30':
      pass
    else:
      buy_data.drop(buy_data[buy_data['Time'] == i].index, inplace = True) 
  buy_data=buy_data.reset_index().drop('index',axis=1)

  data['Open']=buy_data['Open']

  # # END OF MIN DATA

  pandl=[]
  for i in range(0,len(data)):
    if data['indication'][i]=='Buy':
      pandl.append(data['Close'][i] - data['Open'][i])
    else:
      pandl.append(0)
  data['P/L']=pandl
  data=data.drop(['Adj Close','Volume','color','Sash'],axis=1)

  # gain=[]
  # for i in range(0,len(data)):
  #   if data['P/L'][i]==0:
  #     gain.append(0)
  #     continue
  #   if data['stop loss'][i] > data['Close'][i]:
  #     gain.append(data['stop loss'][i] - data['Open'][i])  #remove stoploss
  #   else:
  #     gain.append(data['P/L'][i])
  # data['gain']=gain

  data3=data[data['indication'] == 'Buy']
  # data3=data[data['gain'] != 0]
  data3=data3.reset_index().drop('index',axis=1)

  # stop loss trigger adder

  # gain_correction=[]
  # for i in range(0,len(data3)):
  #   if data3['stop loss'][i] > data3['Low'][i]:
  #     gain_correction.append(data3['stop loss'][i] - data3['Open'][i])
  #   else:
  #     gain_correction.append(data3['gain'][i])

  # data3['gain_correction']=gain_correction


  # data3['INR'] = round(Amount/data3['Open'])*data3['gain_correction']

  data3['INR'] = (round(100000/data3['Open'])) * data3['P/L']

  return sum(data3['INR']),len(data3)

In [ ]:
from tqdm import tqdm
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd '/content/gdrive/MyDrive/SASH/'

/content/gdrive/MyDrive/SASH


In [ ]:
nse=pd.read_csv('ALL_FNO.csv')

In [ ]:
gapup('TCS.NS','2021-02-17','2021-04-11',100000)

(-1616.807861328125, 2)

In [ ]:
bull_sash_stocks={}
for i in tqdm(nse['Symbol']):
    bull_sash_stocks[i]=gapup(i,'2021-02-17','2021-04-11',100000)
final_bull_sash=pd.DataFrame.from_dict(bull_sash_stocks,orient='index',columns=['P/L','counts'],)
final_bull_sash.reset_index(inplace=True)
final_bull_sash=final_bull_sash.rename(columns={'index':'Ticker'})
final_bull_sash.to_csv('bull_gapup_F15_nostoploss.csv')

100%|██████████| 137/137 [13:40<00:00,  5.99s/it]
